In [268]:
import numpy as np
import scipy as sp

In [269]:
def read_input(file_name):
    with open(file_name) as f:
        elfs = []
        for y, line in enumerate(f.readlines()):
            line = line.strip()
            if line == "":
                continue
            elfs.extend((x, y) for x, c in enumerate(line) if c == "#")
    return np.array(elfs)

In [293]:
m = read_input("example")
m

array([[7, 2],
       [5, 3],
       [6, 3],
       [7, 3],
       [9, 3],
       [3, 4],
       [7, 4],
       [9, 4],
       [4, 5],
       [8, 5],
       [9, 5],
       [3, 6],
       [5, 6],
       [6, 6],
       [7, 6],
       [3, 7],
       [4, 7],
       [6, 7],
       [8, 7],
       [9, 7],
       [4, 8],
       [7, 8]])

In [294]:
m.max(axis=0)

array([9, 8])

In [295]:
def draw_map(m):
    x0, y0 = m.min(axis=0)
    x1, y1 = m.max(axis=0)
    lines = []
    for y in range(y0, y1+1):
        lines.append(['.'] * (x1 - x0 + 1))

    for x, y in m:
        lines[y - y0][x - x0] = "#"

    for line in lines:
        print("".join(line))

In [296]:
draw_map(m)

....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..


In [297]:
dx = np.array((1, 0))
dy = np.array((0, 1))

In [298]:
# nearest neighbours of one elf
elf_num = 6
nn = m[sp.spatial.distance_matrix(m, m, p=2)[elf_num]<2]
m[elf_num], nn

(array([7, 4]),
 array([[6, 3],
        [7, 3],
        [7, 4],
        [8, 5]]))

In [299]:
# of these how are there any if we make a step?
res = nn[(sp.spatial.distance_matrix([m[elf_num]-dy], nn, p=2)<=1)[0]]
res, len(res)

(array([[6, 3],
        [7, 3],
        [7, 4]]),
 3)

In [300]:
moves = [
    ('north', lambda v: v - dy),
    ('south', lambda v: v + dy),
    ('west', lambda v: v - dx),
    ('east', lambda v: v + dx)
]

def try_moves(elfs, moves):
    # nearest neighbours
    dst = sp.spatial.distance_matrix(elfs, elfs, p=2)
    new_elfs = []
    for i in range(len(elfs)):
        nn = elfs[dst[i]<2]
        if len(nn) == 1:
            #print("elf %d has no neighbours and does not move" % i)
            new_elfs.append(elfs[i])
            continue
        open_direction = None
        for direction, mv in moves:
            pos = mv(elfs[i])
            nbs = nn[(sp.spatial.distance_matrix([pos], nn, p=2)<=1)[0]]
            if len(nbs) == 1:
                open_direction = direction
                break
        if open_direction:
            #print("Elf %d can move %s" % (i, open_direction))
            new_elfs.append(pos)
        else:
            new_elfs.append(elfs[i])
            #print("Elf %d cannot move" % (i))
    return np.array(new_elfs)


In [301]:
m2 = try_moves(m, moves)

# find the ones who will end up in the same position
mask = np.sum(sp.spatial.distance_matrix(m2, m2, p=2) == 0, axis=1) > 1
# they will not move!
m2[mask] = m[mask]

In [302]:
draw_map(m2)

.....#...
...#...#.
.#..#.#..
.....#..#
..#.#.##.
#..#.#...
#.#.#.##.
.........
..#..#...


In [303]:
def take_moves(elfs, moves, num_rounds=10):
    moves = list(moves) # take copy
    m = elfs
    for i in range(num_rounds):
        print(i, moves[0])
        m2 = try_moves(m, moves)

        # find the ones who will end up in the same position
        mask = np.sum(sp.spatial.distance_matrix(m2, m2, p=2) == 0, axis=1) > 1
        # they will not move!
        m2[mask] = m[mask]

        moves = moves[1:] + [moves[0]]
        m = m2
    return m

In [304]:
m_10 = take_moves(m, moves, num_rounds=10)
draw_map(m_10)

0 ('north', <function <lambda> at 0x7ff761afbaf0>)
1 ('south', <function <lambda> at 0x7ff761afba60>)
2 ('west', <function <lambda> at 0x7ff761afb5e0>)
3 ('east', <function <lambda> at 0x7ff761afb670>)
4 ('north', <function <lambda> at 0x7ff761afbaf0>)
5 ('south', <function <lambda> at 0x7ff761afba60>)
6 ('west', <function <lambda> at 0x7ff761afb5e0>)
7 ('east', <function <lambda> at 0x7ff761afb670>)
8 ('north', <function <lambda> at 0x7ff761afbaf0>)
9 ('south', <function <lambda> at 0x7ff761afba60>)
......#.....
..........#.
.#.#..#.....
.....#......
..#.....#..#
#......##...
....##......
.#........#.
...#.#..#...
............
...#..#..#..


In [305]:
x0, y0 = m_10.min(axis=0)
x1, y1 = m_10.max(axis=0)
(x1-x0)*(y1-y0), (x1 - x0 +1)*(y1 - y0 + 1) - len(m)

(110, 110)

Once again a tricky mistake. I did not read the instructions carefully 
enough and multiplying just 

$$(x_1 - x_0)(y_1 - y_0)$$ 

gave the correct number too.But we have to calulate 

$$
(x_1 - x_0 + 1)(y_1 - y_0 + 1) - N_\textrm{elf} \, .
$$

# Part 1

In [306]:
m = read_input("input")

In [307]:
moves

[('north', <function __main__.<lambda>(v)>),
 ('south', <function __main__.<lambda>(v)>),
 ('west', <function __main__.<lambda>(v)>),
 ('east', <function __main__.<lambda>(v)>)]

In [308]:
m_10 = take_moves(m, moves, num_rounds=10)

0 ('north', <function <lambda> at 0x7ff761afbaf0>)
1 ('south', <function <lambda> at 0x7ff761afba60>)
2 ('west', <function <lambda> at 0x7ff761afb5e0>)
3 ('east', <function <lambda> at 0x7ff761afb670>)
4 ('north', <function <lambda> at 0x7ff761afbaf0>)
5 ('south', <function <lambda> at 0x7ff761afba60>)
6 ('west', <function <lambda> at 0x7ff761afb5e0>)
7 ('east', <function <lambda> at 0x7ff761afb670>)
8 ('north', <function <lambda> at 0x7ff761afbaf0>)
9 ('south', <function <lambda> at 0x7ff761afba60>)


In [309]:
x0, y0 = m_10.min(axis=0)
x1, y1 = m_10.max(axis=0)
(x1-x0)*(y1-y0), (x1 - x0 +1)*(y1 - y0 + 1) - len(m)

(6889, 4302)

```
That's not the right answer; your answer is too high. If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit. Please wait one minute before trying again. (You guessed 6889.) [Return to Day 23]
```

# Part 2

In [339]:
def take_moves2(elfs, moves, num_rounds=10):
    moves = list(moves) # take copy
    m = elfs
    for i in range(num_rounds):
        print(i, moves[0][0], end=", ")
        m2 = try_moves(m, moves)

        # find the ones who will end up in the same position
        mask = np.sum(sp.spatial.distance_matrix(m2, m2, p=2) == 0, axis=1) > 1
        # they will not move!
        m2[mask] = m[mask]

        moves = moves[1:] + [moves[0]]
        # did any elf move?
        num_moved = np.sum(m != m2)
        print("moved elfs: %d" % num_moved)
        if num_moved == 0:
            break
        m = m2
    return m, i

In [340]:
m = read_input("example")

In [341]:
m_10 = take_moves2(m, moves, 30)

0 north, moved elfs: 11
1 south, moved elfs: 11
2 west, moved elfs: 13
3 east, moved elfs: 14
4 north, moved elfs: 19
5 south, moved elfs: 8
6 west, moved elfs: 10
7 east, moved elfs: 11
8 north, moved elfs: 6
9 south, moved elfs: 9
10 west, moved elfs: 7
11 east, moved elfs: 6
12 north, moved elfs: 5
13 south, moved elfs: 6
14 west, moved elfs: 6
15 east, moved elfs: 4
16 north, moved elfs: 2
17 south, moved elfs: 2
18 west, moved elfs: 2
19 east, moved elfs: 0


In [345]:
moves = [
    ('north', lambda v: v - dy),
    ('south', lambda v: v + dy),
    ('west', lambda v: v - dx),
    ('east', lambda v: v + dx)
]

In [346]:
m = read_input("input")

In [347]:
m_10 = take_moves2(m, moves, 10000)

0 north, moved elfs: 756
1 south, moved elfs: 581
2 west, moved elfs: 494
3 east, moved elfs: 464
4 north, moved elfs: 467
5 south, moved elfs: 475
6 west, moved elfs: 434
7 east, moved elfs: 445
8 north, moved elfs: 451
9 south, moved elfs: 462
10 west, moved elfs: 475
11 east, moved elfs: 503
12 north, moved elfs: 493
13 south, moved elfs: 521
14 west, moved elfs: 496
15 east, moved elfs: 523
16 north, moved elfs: 499
17 south, moved elfs: 528
18 west, moved elfs: 504
19 east, moved elfs: 543
20 north, moved elfs: 543
21 south, moved elfs: 574
22 west, moved elfs: 562
23 east, moved elfs: 595
24 north, moved elfs: 550
25 south, moved elfs: 573
26 west, moved elfs: 553
27 east, moved elfs: 587
28 north, moved elfs: 572
29 south, moved elfs: 634
30 west, moved elfs: 592
31 east, moved elfs: 618
32 north, moved elfs: 591
33 south, moved elfs: 640
34 west, moved elfs: 638
35 east, moved elfs: 681
36 north, moved elfs: 633
37 south, moved elfs: 664
38 west, moved elfs: 654
39 east, moved 

This took approx 12 minutes on my laptop! Way too long. There are better solutions like:
https://github.com/mjpieters/adventofcode/blob/master/2022/Day%2023.ipynb 